In [1]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [12,8]
matplotlib.rc('font',family='Monospace')

In [2]:
class MDARP(object):
    """
    multi-dimensional autoregressive process with several noise terms
    """
    def __init__(self,**kwargs):
        # construct matrix A from eigenvalues and angle (0...1) between eigenvectors, assume first direction is (0,1)
        self.__A_eigval    = np.array([self.interval(x) for x in kwargs.get('eigval',[0.9,0.5])],dtype=np.float)
        self.__A_angle     = self.interval(kwargs.get('A_angle',0.33)) # angles between 0 .. 1
        evec1              = np.array([0,1],dtype=np.float)
        evec2              = np.dot(self.rotation(self.__A_angle),evec1)
        self.__A_eigvec    = (evec1 / np.linalg.norm(evec1), evec2 / np.linalg.norm(evec2))
        evmat              = np.array(self.__A_eigvec,dtype=float).T
        self.A             = np.matmul(np.matmul(evmat,np.diag(self.__A_eigval)), np.linalg.inv(evmat))
        
        # projection vector is also with respect to first EVec
        self.__alpha_angle = self.interval(kwargs.get('alpha_angle',0.67))
        self.alpha         = np.dot(self.rotation(self.__alpha_angle),evec1)

        # noise, env
        self.noiseamplitudes = np.array(kwargs.get('noiseamplitudes',[1/np.sqrt(2.),1/np.sqrt(2.)]),dtype=np.float)

        self.experimenttype = kwargs.get('experimenttype','sisters')
        self.default_steps  = kwargs.get('steps',10)
        
        # initialize all dynamical variables to start
        self.reset()


    def random(self,mean = 0, sqrt_var = 1):
        return np.random.normal(loc = mean, scale = sqrt_var, size=2)


    def step(self):
        # noiseamplitudes = noise 0, env 1
        noiseA          = self.noiseamplitudes[0] * self.random()
        noiseB          = self.noiseamplitudes[0] * self.random()
        if self.experimenttype == 'sisters' or self.experimenttype == 'nonsisters':
            xiE         = self.random()
            noiseA     += self.noiseamplitudes[1] * xiE
            noiseB     += self.noiseamplitudes[1] * xiE
        else:
            noiseA     += self.noiseamplitudes[1] * self.random()
            noiseB     += self.noiseamplitudes[1] * self.random()

        xA_new = np.dot(self.A,self.xA[-1]) + noiseA
        xB_new = np.dot(self.A,self.xB[-1]) + noiseB
        
        self.xA = np.vstack((self.xA,[xA_new]))
        self.xB = np.vstack((self.xB,[xB_new]))
        
        self.__current_generation += 1
        
        return xA_new, xB_new

    def reset(self):
        self.__current_generation = 0
        if self.experimenttype == 'sisters':
            start   = self.random()
            self.xA = np.array([start])
            self.xB = np.array([start])
        else:
            self.xA = np.array([self.random()])
            self.xB = np.array([self.random()])
        

    def run(self,steps = None):
        self.reset()
        if steps is None:
            steps = self.default_steps
            
        for i in np.arange(steps):
            self.step()
        return self.xA, self.xB

        
    def rotation(self,angle):
        return np.array([[np.cos(2*np.pi*angle),np.sin(2*np.pi*angle)],[-np.sin(2*np.pi*angle),np.cos(2*np.pi*angle)]],dtype=np.float)


    def interval(self,value,minval = 0,maxval = 1):
        step = maxval - minval
        cval = value
        while cval < minval:cval += step
        while cval > maxval:cval -= step
        return cval


    def projection(self,x, alphaangle = None):
        if alphaangle is None:  alphavec = self.alpha
        else:                   alphavec = np.dot(self.rotation(alphaangle),self.__A_eigvec[0])
        
        alphavec = alphavec/np.linalg.norm(alphavec)
        return np.dot(alphavec,x)
    
    
    def output(self,step = None,alphaangle = None):
        if step is None:    step = -1
        else:               step = np.max([0,np.min([len(self.xA)-1,step])])
        return self.projection(self.xA[step],alphaangle), self.projection(self.xB[step],alphaangle)


    def output_all(self,alphaangle = None):
        return np.array([self.output(step = i,alphaangle = alphaangle) for i in range(len(self))]).T
    
    
    def __len__(self):
        assert len(self.xA) == len(self.xB)
        return len(self.xA)


In [3]:
generations = 50
g = np.arange(generations+1)
colors =[['fcaf3e','ce5c00'],['729fcf','204a87'],['8ae234','4e9a06'],['fce94f','c4a000'], ['ad7fa8','5c3566'],['ef2929','a40000'],['e9b96e','8f5902']]
@interact
def InteractiveWindow(numproc = (1,10,1), lambda1 = (0,1.), lambda2 = (0,1.), Aangle = (0,1.), alpha = (0,1.), noise1 = (0,2.), noise2 = (0,2.), experimenttype = ['sisters','nonsisters','control']):
    p = [MDARP(eigval = [lambda1,lambda2], A_angle = Aangle, noiseamplitudes = [noise1,noise2], alpha_angle = alpha, experimenttype = experimenttype, steps = generations) for i in range(numproc)]
    for i in range(numproc):
        p[i].run()
        traj = p[i].output_all()
        plt.plot(g, traj[0], c = '#{}'.format(colors[i % len(colors)][0]))
        plt.plot(g, traj[1], c = '#{}'.format(colors[i % len(colors)][1]))

interactive(children=(IntSlider(value=5, description='numproc', max=10, min=1), FloatSlider(value=0.5, descrip…

In [ ]:
@interact
def InteractiveWindow(numproc = (0,200,10), lambda1 = (0,1.), lambda2 = (0,1.), Aangle = (0,1.), alpha = (0,1.), noise1 = (0,2.), noise2 = (0,2.), yzoom=(0,10,1)):
    for experimenttype in  ['sisters','nonsisters','control']:
        p = [MDARP(eigval = [lambda1,lambda2], A_angle = Aangle, noiseamplitudes = [noise1,noise2], alpha_angle = alpha, experimenttype = experimenttype, steps = generations) for i in range(numproc)]
        sum_dt2 = np.zeros(generations+1)
        for i in range(numproc):
            p[i].run()
            traj = p[i].output_all()

            dt = np.array([np.sum(traj[0][:j]-traj[1][:j]) for j in range(generations+1)])

            #print(dt)
            if i == 0:
                all_dt = np.array([dt])
            else:
                all_dt = np.vstack([all_dt,[dt]])

            #sum_dt2 += np.array([np.sum((traj[0][:j]-traj[1][:j])**2) for j in range(generations+1)])

        plt.plot(g,np.var(all_dt,axis = 0))
    plt.ylim((0,10**yzoom))
    plt.legend( ['sisters','nonsisters','control'])
    
    
    
    